In [ ]:
!pip install torch==1.1.0
!pip install git+https://github.com/huggingface/transformers.git@7b75aa9fa55bee577e2c7403301ed31103125a35

In [ ]:
!git clone https://github.com/Senyu-T/unifiedqa

In [6]:
cd unifiedqa/bart

/content/unifiedqa/bart


In [7]:
%ls

bart.py  data.py            README.md  unified_data.py
cli.py*  download_data.sh*  run.py


In [ ]:
!chmod +x download_data.sh; ./download_data.sh

In [9]:
ls

bart.py  data/    download_data.sh*  run.py
cli.py*  data.py  README.md          unified_data.py


In [10]:
!python cli.py --do_train --output_dir out/finetune_bart_base_2_epoch \
    --train_file data/natural_questions_with_dpr_para/train.tsv \
    --predict_file data/natural_questions_with_dpr_para/dev.tsv \
    --train_batch_size 16\
    --predict_batch_size 16\
    --append_another_bos --do_lowercase --num_train_epochs 2 --verbose

2021-04-15 03:02:24.959688: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
04/15/2021 03:02:27 - INFO - __main__ - Namespace(adam_epsilon=1e-08, append_another_bos=True, checkpoint=None, checkpoint_step=0, debug=False, do_lowercase=True, do_predict=False, do_train=True, eval_period=2000, gradient_accumulation_steps=1, is_unifiedqa=False, learning_rate=1e-05, max_grad_norm=1.0, max_input_length=512, max_output_length=100, num_beams=4, num_train_epochs=2.0, output_dir='out/finetune_bart_base_2_epoch', predict_batch_size=16, predict_file='data/natural_questions_with_dpr_para/dev.tsv', prefix='', seed=42, skip_inference=False, train_batch_size=16, train_file='data/natural_questions_with_dpr_para/train.tsv', verbose=True, wait_step=10, warmup_proportion=0.01, warmup_steps=0, weight_decay=0.0)
04/15/2021 03:02:27 - INFO - __main__ - out/finetune_bart_base_2_epoch
04/15/2021 03:02:27 - INFO - __main__ - Using 1 gpus
04/15/

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [36]:
cp -r out ../../drive/MyDrive

In [37]:
!nvidia-smi -L

GPU 0: Tesla V100-SXM2-16GB (UUID: GPU-8cc1ac53-77a9-9d54-5e19-5f8428ce32c8)


In [35]:
!python cli.py --do_predict --output_dir out/finetune_bart_base_2_epoch \
    --predict_file data/natural_questions_with_dpr_para/dev.tsv \
    --predict_batch_size 16\
    --append_another_bos

2021-04-15 06:05:20.271987: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Output directory () already exists and is not empty.
04/15/2021 06:05:22 - INFO - __main__ - Namespace(adam_epsilon=1e-08, append_another_bos=True, checkpoint=None, checkpoint_step=0, debug=False, do_lowercase=False, do_predict=True, do_train=False, eval_period=2000, gradient_accumulation_steps=1, is_unifiedqa=False, learning_rate=1e-05, max_grad_norm=1.0, max_input_length=512, max_output_length=100, num_beams=4, num_train_epochs=10000.0, output_dir='out/finetune_bart_base_2_epoch', predict_batch_size=16, predict_file='data/natural_questions_with_dpr_para/dev.tsv', prefix='', seed=42, skip_inference=False, train_batch_size=40, train_file='data/train.tsv', verbose=False, wait_step=10, warmup_proportion=0.01, warmup_steps=0, weight_decay=0.0)
04/15/2021 06:05:22 - INFO - __main__ - out/finetune_bart_base_2_epoch
04/15/2021 06:05:22 - INFO - __m

Now we have the model output. We also find that the "nq-dev" set provided by DPR is a subset of current dev set, so in order to reproduce results in unitedQA we'll use the dev set in this repo which contains 10000 questions.

In [114]:
cd /content/unifiedqa/bart/data/natural_questions_with_dpr_para/

/content/unifiedqa/bart/data/natural_questions_with_dpr_para


In [136]:
# read dev file, parse into context / question / answer for further data analysis
infer_answers = []
infer_questions = []
infer_contexts = []
with open('dev.tsv', 'rb') as inference_in:
    lines = inference_in.readlines()
    for i in range(len(lines)):
        sep = str(lines[i]).split('\\n') 
        infer_questions.append(sep[0][2:-1])
        infer_answers.append((sep[1].split('\\t')[-1]).lower())
        infer_contexts.append(sep[1].split('\\t')[0])



In [51]:
cd /content/unifiedqa/bart/out/finetune_bart_base_2_epoch

/content/unifiedqa/bart/out/finetune_bart_base_2_epoch


In [44]:
ls

best-model.pt  eval_log.txt  log.txt  predictions.json


In [ ]:
# read the prediction file output by model
import json
with open("predictions.json", 'rb') as m_out:
    pred = json.load(m_out)


In [66]:
# check how many short answers are not extracted from the context
not_in_dpr_idx = []
for i in range(len(infer_answers)):
  if not infer_answers[i] in infer_contexts[i]:
    not_in_dpr_idx.append(i)

In [62]:
print(len(infer_answers), len(pred))

10693 10693


In [153]:
cd /content/unifiedqa/bart

/content/unifiedqa/bart


In [154]:
def store_file(out_dir, arr):
  with open(out_dir, 'w') as f:
    f.write("\n".join(str(arr[i]) for i in range(len(arr))))

In [87]:
store_file("pred.txt", pred)

In [152]:
# detokenize: restore special marks
# also find special token (forien language)
cnt_special_char = 0
for i in range(len(infer_answers)):
  infer_answers[i] = infer_answers[i].replace(' \\\'\\\'', ' \'\'')   # double quotation
  infer_answers[i] = infer_answers[i].replace('\\\'', '\'')
  infer_answers[i] = infer_answers[i].replace(' \'s', '\'s')    # 's
  infer_answers[i] = infer_answers[i].replace(' ,', ',')        # comma
  if '\\' in infer_answers[i]:
    cnt_special_char += 1
    #print(f"{i}  {infer_answers[i]}      {pred[i]}")
print(cnt_special_char)

 

120


In [155]:
store_file("infer_ans.txt", infer_answers)

In [159]:
cp infer_ans.txt /content/drive/MyDrive

In [160]:
# Evaluate 
is_correct = 0.0
for i in range(len(infer_answers)):
  if infer_answers[i] == pred[i]:
    #print(pred[i], i)
    is_correct += 1

print(is_correct / len(infer_answers))

0.2904704011970448


In [162]:
print(is_correct / (len(infer_answers) - cnt_special_char))

0.2937671427220278


In [165]:
with open("error.txt", "w") as f:
  for i in range(len(infer_answers)):
    if infer_answers[i] != pred[i]:
      f.write(f"{i}: {infer_answers[i]:30s}{pred[i]}\n")

In [167]:
cp error.txt /content/drive/MyDrive